In [84]:
import  string
import pandas as pd
file_path = "sample_case.txt"
TO_DELETE = string.punctuation

    
def remove_punctuations(__text):
    translator =  str.maketrans('', '', TO_DELETE)
    return __text.translate(translator)
    

with open(file_path, 'r') as file_ptr:        
    text = file_ptr.read()
    file_ptr.close()

processed_text =  remove_punctuations(text)
print(processed_text)


Court Seal
District Court of Financial Frauds
123 Main Street Anytown USA
Date October 19 2023

Case No 2023123
Plaintiff John Smith
Defendant Sarah Johnson

JUDGMENT

In the matter of the case regarding financial fraud brought before this honorable court having reviewed the evidence and testimony presented by both parties and considered the arguments put forth by their respective legal representatives the court hereby issues the following judgment

Background
The plaintiff John Smith filed a complaint against the defendant Sarah Johnson accusing her of financial fraud The plaintiff alleged that the defendant engaged in a series of fraudulent financial activities including embezzlement misappropriation of funds and deceptive accounting practices

Findings
After careful consideration of the evidence presented during the trial this court finds the following

1 Misappropriation of Funds The defendant Sarah Johnson has been found guilty of misappropriating funds totaling 500000 from XYZ Co

In [65]:
import spacy
from spacy import displacy

from tqdm import tqdm
from textblob import  TextBlob


model = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

doc = nlp(processed_text)

entities = []

print("Entities:")
for ent in doc.ents:
    entities.append(
        (ent.text, ent.label_)
    )
entities_data_frame = pd.DataFrame(
    entities,
    columns=["Entity", "Label"]
)
display(entities_data_frame)

# Extract and print relationships between entities
print("\nRelationships:")
subject_action_pairs = []
for token in doc:
    if token.dep_ == "nsubj":
        subject = token.text
        action = token.head.text
        subject_action_pairs.append(
            (subject, action)
        )
data_frame = pd.DataFrame(
    subject_action_pairs,
    columns=["Subject", "Action"]
)

data_frame




Entities:


,Entity,Label
0,District Court of Financial Frauds,ORG
1,123,CARDINAL
2,USA,GPE
3,John Smith,PERSON
4,Sarah Johnson,PERSON
5,John Smith,PERSON
6,Sarah Johnson,PERSON
7,Findings\nAfter,WORK_OF_ART
8,1,CARDINAL
9,Sarah Johnson,PERSON



Relationships:


,Subject,Action
0,court,reviewed
1,arguments,put
2,court,issues
3,Johnson,accusing
4,plaintiff,alleged
5,defendant,engaged
6,court,finds
7,evidence,supports
8,evidence,shows
9,defendant,diverted


In [75]:
#EXTRACT THE Decision  part in a judgement FILE.
from spacy.matcher import Matcher


matcher = Matcher(nlp.vocab)
assert len(matcher) == 0
matcher.add("Rule", [[{"ORTH": "Decision"}]])
assert len(matcher) == 1

pattern = [{"LOWER": "finding"}, {"LOWER": "findings"}]
matcher.add("Find_Findings", [pattern])

matches = matcher(doc)

print(
    doc[matches[0][1]]
)



Decision


In [67]:
from spacy.matcher import Matcher

sample_text = "Kelvin Kering printed a good document."

nlp_processor = spacy.load("en_core_web_sm")

matcher = Matcher(nlp_processor.vocab)
prop_noun_verb_pattern = [
    {"POS": "PROPN", "OP": "+"},
    {"POS": "VERB"}
]
matcher.add("PROP_NOUN_FOLLOWED_BY_VERB", [prop_noun_verb_pattern], greedy="LONGEST")

doc2 = nlp_processor(processed_text)
matches = matcher(doc2)
matches.sort(key=lambda match: match[1])
print(len(matches))
prop_noun_list = []
#tNote: lambdas are used in the filter and the sort methods .The filter means filter_in (return if the following conditions are  emt ...).
for match_tuple in matches[:10]:
    
    span_token=doc2[match_tuple[1]:match_tuple[2]]
    # print(type(span_token)) -------> outputs spacy.tokens.span.Span ... Spacy doc is a special list , with word tokens, span tokens and sentences.
    
    prop_noun_list.append(
        (match_tuple[0], match_tuple[1], match_tuple[2],span_token.text)
    )
    
prop_noun_df = pd.DataFrame(
    prop_noun_list,
    columns=["VocabLexeme", "StartIndex", "EndIndex", "Text"]
)
prop_noun_df



2


,VocabLexeme,StartIndex,EndIndex,Text
0,14035105279653467510,81,84,John Smith filed
1,14035105279653467510,89,92,Sarah Johnson accusing


In [87]:
#CHALLENGE 3 :  extracting text from the start of one text to another 
quot_nlp = spacy.load("en_core_web_sm")
quot_file_name = "sample_quotation_test.txt"
speak_lemmas = ["speak","say", "ask"]

with open(quot_file_name, 'r') as file_pointer:        
    quotation_text = file_pointer.read()

matcher = Matcher(nlp.vocab)
quotation_pattern = [
    {"ORTH": "'"},
    {"IS_ALPHA": True, "OP":"+"},
    {"IS_PUNCT": True, "OP":"*"},
    {"ORTH": "'"},
    {"POS": "VERB", "LEMMA":{
        "IN":speak_lemmas
    }},
    {"POS": "PROPN", "OP":"+"}
]
matcher.add("QUOTATION_PATTERN", [quotation_pattern], greedy="LONGEST")

quot_doc = quot_nlp(quotation_text)

quot_matches_list= matcher(quot_doc)
quot_matches_list.sort(key=lambda match: match[1])

quot_list = []

for match_id, token_start, token_end in quot_matches_list:
    span = quot_doc[token_start:token_end]
    quot_list.append((match_id,span.text))
    
quot_data_frame = pd.DataFrame(
    quot_list, 
    columns=["Match_Id","Quotation"]
)
display(quot_data_frame) 


,Match_Id,Quotation
0,13254208182742973300,"'This is indeed a fake product !,' said Alice"


Mary PERSON


In [ ]:
#CUSTOM COMPONENTS 
from spacy.language  import Language
@Language.component("remove_gpe")
def remove_gpe(doc):
    original_ents = list(doc.ents)
    for ent in doc.ents:
        if ent.label_ =="GPE":
            original_ents.remove(ent)
    doc.ents = original_ents
    return doc
    
quot_nlp.add_pipe("remove_gpe")
nlp.analyze_pipes()
test_text = "Kenya is a good nation . Mary lives in Britain however"
test_doc = quot_nlp(test_text)
for ent in test_doc.ents:
    print(ent.text, ent.label_)